https://github.com/FrankBGao/HeatMap_for_TuShare

In [1]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.embed import components
from datetime import datetime

In [4]:
def get_index(data):
    data['index_len'] = range(len(data))
    return data

def get_heatmap(day,one_day):
    one_day = one_day.drop_duplicates()
    print (len(one_day))
    # I don't want 创业板
    one_day['code_is'] = one_day['code'].apply(lambda x: x[0])
    one_day = one_day[one_day['code_is'] != '3']

    one_day = one_day.groupby('c_name', as_index=False).apply(get_index)
    romans = one_day['c_name'].drop_duplicates().values  # [str(x) for x in range(0, one_day['index_len'].max())]


    group_range = [str(x) for x in range(0, one_day['index_len'].max())]

    colormap = {
        -11: '#005824',
        -10: '#005824',
        -9: '#1A693B',
        -8: '#347B53',
        -7: '#4F8D6B',
        -6: '#699F83',
        -5: '#83B09B',
        -4: '#9EC2B3',
        -3: '#B8D4CB',
        -2: '#D2E6E3',
        -1: '#EDF8FB',
        0: '#ededed',
        1: '#ffcfdc',
        2: '#ffcccc',
        3: '#ff8080',
        4: '#ff5959',
        5: '#ff4040',
        6: '#d90000',
        7: '#b20000',
        8: '#7f0000',
        9: '#660000',
        10: '#400000',
        11: '#400000'
    }
    if 'trade' in one_day.columns:
        source = ColumnDataSource(
            data=dict(
                group=[unicode(x) for x in one_day["c_name"]],
                period=[unicode(y) for y in one_day["index_len"]],
                type_color=[colormap[x] for x in one_day["rate"]],
                code=one_day["code"],
                amount=one_day["amount"],
                trade=one_day["trade"],
                c_name=one_day["c_name"],
                name=one_day["name"],
                rate=one_day["rate"],
                volume=one_day["volume"],
                high=one_day["high"],
                low=one_day["low"],
                open=one_day["open"],
            )
        )
        p = figure(title=u"今日大盘行情"+ str(datetime.now()) , tools="pan,wheel_zoom,box_zoom,reset,save,hover",
                   x_range=group_range, y_range=list(reversed(romans)))
    else:
        source = ColumnDataSource(
            data=dict(
                group=[unicode(x) for x in one_day["c_name"]],
                period=[unicode(y) for y in one_day["index_len"]],
                type_color=[colormap[x] for x in one_day["rate"]],
                code=one_day["code"],
                amount=one_day["amount"],
                close=one_day["close"],
                c_name=one_day["c_name"],
                name=one_day["name"],
                rate=one_day["rate"],
                volume=one_day["volume"],
                high=one_day["high"],
                low=one_day["low"],
                open=one_day["open"],
            )
        )
        p = figure(title=u"大盘行情" + day, tools="pan,wheel_zoom,box_zoom,reset,save,hover",
                   x_range=group_range, y_range=list(reversed(romans)))


    p.plot_width = 1200
    p.toolbar_location = None
    p.outline_line_color = None

    p.rect("period", "group", 0.9, 0.9, source=source,
           fill_alpha=0.6, color="type_color")

    p.select_one(HoverTool).tooltips = [
        ("c_name", "@c_name"),
        ("name", "@name"),
        ("code", "@code"),
        ("rate", "@rate"),
        ("volume", "@volume"),
        ("amount", "@amount"),
        ("high", "@high"),
        ("low", "@low"),
        ("open", "@open"),
        ("close", "@close"),
        ("trade", "@trade"),
    ]
    script, div = components(p)
    return div + script

In [5]:
# -*- coding: utf8 -*-
import pandas as pd

def too_big_get_11(data):
    if data > 10:
        return 11
    elif  data < -10:
        return -11
    else:
        return data


def get_data_real_time(code_info,today):
    today = today.drop('name',axis = 1)
    today['rate'] = today['changepercent']

    data = pd.merge(today, code_info, on='code')

    data['rate'] = data['rate'].apply(too_big_get_11)
    data['rate'] = data['rate'].astype(int)
    return data

In [6]:
from tornado.httpserver import HTTPServer
from tornado.ioloop import IOLoop
from tornado.wsgi import WSGIContainer
from stock_price import app


server = HTTPServer(WSGIContainer(app))
server.listen(5001)
IOLoop.current().start()

ModuleNotFoundError: No module named 'stock_price'

# stock_price.py

In [ ]:
from flask import Flask,  request,  render_template
import tushare as ts
import get_data as gd
import heatmap_real_time as heatmap
app = Flask(__name__)


@app.route('/', methods=['GET'])
def plot_days():
    if request.method == 'GET' :
        today = ts.get_today_all()
        code_info = ts.get_industry_classified()

        today['code'] = today['code'].astype(unicode)
        one_day = gd.get_data_real_time(code_info, today)
        body = heatmap.get_heatmap('Today', one_day)
        return render_template('heatmap.html', body=body)

if __name__ == '__main__':
    app.debug = True
    app.run()